# Capstone Project: Feature Selection and Engineering 

### Imports

In [1]:
import pandas as pd
import numpy as np
import geopy.distance
import numbers
from functools import reduce

### Dataset

In [2]:
capstone_df = pd.read_csv("../data/location_with_google_ratings.csv")

In [3]:
capstone_df.head()

,area,county,city_or_town,lat_area,long_area,population,median_household_income,income_per_capita,name,address,lat_venue,long_venue,categories,price_level,rating,user_ratings_total
0,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,Chuck Corica Golf Complex,1 Clubhouse Memorial Rd,37.738885,-122.234326,Golf Course,0.0,0.0,0.0
1,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,Shoreline Park,"Alameda, CA",37.742580,-122.250126,Park,0.0,0.0,0.0
2,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,Angel Fish Japanese Cuisine,883 C-2 Island Dr,37.737478,-122.240107,Japanese Restaurant,2.0,4.3,112.0
3,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,La Penca Azul,891 Island Dr Ste B (at Mecartney Rd),37.737667,-122.240767,Mexican Restaurant,2.0,4.3,2263.0
4,Alameda,Alameda,City,37.734379,-122.23952,77624,104756,52448,La Val's Pizza,891 Island Dr,37.737610,-122.241001,Italian Restaurant,2.0,4.4,190.0


Based on the EDA and understanding of the problem statement, we select features that would be used within our model and inference, which are:

- Population and Density Metrics:
    - User Ratings as % of Population: No. of User Ratings here are used as a proxy for traffic in the area and is engineered to become a percent of the population
    - Venue and Coffee Shop per Capita: Number of venues and coffee shops divided by the population per area
    
- Distance Matrix
    - Venue per Area: Calculating the distance (in meters) between each venue
    - Average Distance per Category per Area: For every venue the minimum distance between said venue to another venue of each category are extrapolated
    
- Category Frequency per Area
    - Feature is engineered by dividing the number of venues per category per area by the number of venues per category

Two datasets are then generated for modeling purposes:
1. Venue per area (kept separate because it would make the csv file even bigger)
2. Dataframe with the features mentioned above combined together aside from Venue per Area

*Note: To test out the code, a similar process was applied onto SF only venues which can be seen in the [Appendix](./6.01-appendix_feature_engineering_SF_only.ipynb)*

### Population and Density Metrics

#### User Rating as % of Population

In [4]:
user_as_pct_pop = capstone_df[["name","categories","area","lat_venue","long_venue","population","user_ratings_total"]]

In [5]:
user_as_pct_pop["user_rate_as_pct_pop"] = user_as_pct_pop["user_ratings_total"] / user_as_pct_pop["population"]

<ipython-input-5-a4965d6fb73e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_as_pct_pop["user_rate_as_pct_pop"] = user_as_pct_pop["user_ratings_total"] / user_as_pct_pop["population"]


In [6]:
user_as_pct_pop = pd.DataFrame(user_as_pct_pop.groupby(["area","categories"])["user_rate_as_pct_pop"].mean())

In [7]:
user_as_pct_pop.head()

user_rate_as_pct_pop
area    categories                               
Alameda Afghan Restaurant                0.007021
        Airport Service                  0.000000
        American Restaurant              0.006995
        Arcade                           0.000000
        Bakery                           0.000253

#### Venue and Coffee Shop per Capita

In [8]:
per_cap = capstone_df[["area","name","lat_venue","long_venue","population","categories"]]

In [9]:
per_cap["id"] = per_cap["name"]+per_cap["area"]+per_cap["categories"]+str(per_cap["lat_venue"])+str(per_cap["long_venue"])

<ipython-input-9-c34216206e3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per_cap["id"] = per_cap["name"]+per_cap["area"]+per_cap["categories"]+str(per_cap["lat_venue"])+str(per_cap["long_venue"])


In [10]:
ven_per_cap = pd.DataFrame(per_cap.groupby(["area","population"])["id"].count())

In [11]:
ven_per_cap["venues_per_capita"] = ven_per_cap["id"] / ven_per_cap.index.get_level_values("population")

In [12]:
print(ven_per_cap.shape)
ven_per_cap.head()

(101, 2)


,,id,venues_per_capita
area,population,,
Alameda,77624,100,0.001288
Albany,19696,50,0.002539
American Canyon,20475,50,0.002442
Antioch,111502,100,0.000897
Atherton,7137,50,0.007006


In [13]:
cs_per_cap = pd.DataFrame(per_cap[per_cap["categories"] == "Coffee Shop"].groupby(["area","population"])["id"].count())

In [14]:
cs_per_cap["cs_per_capita"] = cs_per_cap["id"] / cs_per_cap.index.get_level_values("population")

In [15]:
print(cs_per_cap.shape)
cs_per_cap.head()

(96, 2)


,,id,cs_per_capita
area,population,,
Alameda,77624,4,0.000052
Albany,19696,2,0.000102
American Canyon,20475,3,0.000147
Antioch,111502,8,0.000072
Atherton,7137,4,0.000560


### Distance Matrix

In [16]:
# Thanks Gwen for the source!
# Source: https://gis.stackexchange.com/questions/293310/how-to-use-geoseries-distance-to-get-the-right-answer/346064#346064

In [17]:
distance_df = capstone_df[["area","name","lat_venue","long_venue","categories"]]

In [18]:
distance_df["id"] = distance_df["name"] + distance_df["area"] + distance_df["categories"] + str(distance_df["lat_venue"]) + str(distance_df["long_venue"])

<ipython-input-18-c9d4ed189825>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distance_df["id"] = distance_df["name"] + distance_df["area"] + distance_df["categories"] + str(distance_df["lat_venue"]) + str(distance_df["long_venue"])


#### Venue per Area

In [19]:
ven_per_area = pd.DataFrame(distance_df.groupby(["area","name"])[["lat_venue","long_venue"]].mean())

In [20]:
ven_per_area.head()

lat_venue  long_venue
area    name                                                 
Alameda Alameda Beach                   37.760926 -122.269280
        Alameda Dog Park                37.767796 -122.272758
        Alameda Farmers' Market         37.774196 -122.276047
        Alameda Island Brewing Company  37.768483 -122.238984
        Alameda Theatre & Cineplex      37.764680 -122.243946

In [21]:
area_dic = {}

for area in ven_per_area.index.get_level_values("area").unique():
    area_dic[area] = [venue for venue in ven_per_area[ven_per_area.index.get_level_values("area") == area].index.get_level_values("name").unique()]
        

In [22]:
# Thank you Gwen!!

dist_dic = {}

for k,v in area_dic.items():
    for venue in v:
        print(venue + ", " + k)
        ven_dic = {}
        for ven in ven_per_area.T.columns:
            if ven == (k, venue):
                ven_dic[ven] = 0
            else:
                lat_1 = ven_per_area.loc[(k,venue),"lat_venue"]
                long_1 = ven_per_area.loc[(k,venue),"long_venue"]
                lat_2 = ven_per_area.loc[ven,"lat_venue"]
                long_2 = ven_per_area.loc[ven,"long_venue"]
                distance = geopy.distance.geodesic((lat_1,long_1),(lat_2,long_2))
                meters = distance.meters
                ven_dic[ven] = meters
        dist_dic[(k, venue)] = ven_dic   

Alameda Beach, Alameda
Alameda Dog Park, Alameda
Alameda Farmers' Market, Alameda
Alameda Island Brewing Company, Alameda
Alameda Theatre & Cineplex, Alameda
American Oak, Alameda
Angel Fish Japanese Cuisine, Alameda
Angela's, Alameda
Aria supermarket, Alameda
Blue Dot, Alameda
Body Techniques, Alameda
Books Inc, Alameda
Burma Superstar, Alameda
CRAFT Beer & Wine, Alameda
Championship Plaza, Food Trucks, Alameda
Chipotle Mexican Grill, Alameda
Cholita Linda, Alameda
Chuck Corica Golf Complex, Alameda
Coffee and Tea Traders, Alameda
Cookiebar Creamery, Alameda
Crab Cove, Alameda
Daiso Japan, Alameda
Dan's Fresh Produce, Alameda
Encinal Market, Alameda
Encinal Nursery, Alameda
European Wax Center, Alameda
Feel Good Bakery, Alameda
Feel Good Kitchen, Alameda
Forbidden Island, Alameda
Franklin Park, Alameda
Harbor Bay Club, Alameda
High Scores Arcade, Alameda
Ida Krusi Memorial Park, Alameda
In-N-Out Burger, Alameda
JSX (JetsuiteX), Alameda
Jamba Juice, Alameda
Jay's Coffee Teas and Treats

Residence Inn Palo Alto Menlo Park, Atherton
Ristorante Carpaccio, Atherton
Round Table Pizza, Atherton
Saint Frank Coffee, Atherton
Shiok, Atherton
SoulCycle Palo Alto, Atherton
Sports House Indoor Sports and Cafe, Atherton
Sprinkles, Atherton
Sushi Plus, Atherton
SusieCakes, Atherton
Tacos El Grullense E&E, Atherton
The Refuge, Atherton
Trader Joe's, Atherton
Alice's Restaurant, Belmont
Alpha Acid Brewing, Belmont
Applejack's, Belmont
Arata's Pumpkin Farm, Belmont
Belmont Beauty Salon, Belmont
Blue Bottle Coffee, Belmont
Blue House Farm, Belmont
Blue Sky Cafe, Belmont
Bob's Pumpkin Patch, Belmont
Buck's of Woodside, Belmont
Cambowan, Belmont
Chase Bank, Belmont
Club Oracle, Belmont
Cowell Ranch Beach & Trailhead, Belmont
Cowell-Purisima Trailhead, Belmont
Cuban Kitchen, Belmont
Dance VITA, Belmont
Divino, Belmont
Dog Haus Biergarten, Belmont
El Corte de Madera Open Space Preserve, Belmont
Falafelle, Belmont
Fieldwork Brewing Co. Beer Garden, Belmont
Four Corners Farmer's Market, Belm

Caps & Taps, Berkeley
Caravaggio Gelateria Italiana, Berkeley
Casa De Chocolate, Berkeley
Casa Latina Bakery, Berkeley
Cask, Berkeley
Cedar Rose Park, Berkeley
Cha-Ya Vegetarian Japanese Restaurant, Berkeley
Cheese Board Pizza, Berkeley
Cheese Steak Shop, Berkeley
Chez Panisse, Berkeley
Chick-fil-A, Berkeley
China Village Restaurant, Berkeley
Cinnaholic, Berkeley
Claremont Canyon Regional Preserve, Berkeley
ClubSport, Berkeley
Codornices Park, Berkeley
Cole Coffee, Berkeley
Comal, Berkeley
Combat Sports Academy / CSA CrossFit, Berkeley
Comic Ink, Berkeley
CorePower Yoga, Berkeley
Cornerstone, Berkeley
Cost Plus World Market, Berkeley
Cragmont Park, Berkeley
Crixa Cakes, Berkeley
Crossfit Oakland, Berkeley
Cupkates @ Off the Grid: Haight, Berkeley
Curry Kona, Berkeley
César, Berkeley
Dark Carnival Imaginative Fiction Bookstore, Berkeley
De La Torre's Trattoria, Berkeley
Denica's Real Food Kitchen, Berkeley
Donato & Co., Berkeley
DoubleTree by Hilton, Berkeley
Downtown Berkeley Farmers M

Pho Vietnam, Brentwood
Quik Stop, Brentwood
Raley's, Brentwood
Safeway, Brentwood
Shadow Lakes, Brentwood
Smith Family Farms, Brentwood
Sprouts Farmers Market, Brentwood
Starbucks, Brentwood
Subway, Brentwood
Sweeney's Grill & Bar, Brentwood
Taco Bell, Brentwood
The Streets of Brentwood, Brentwood
Trader Joe's, Brentwood
Veterans Park, Brentwood
Victoria's Secret PINK, Brentwood
Willy's Bagels & Blends, Brentwood
Willys Bagels, Brentwood
Yogurt Pizazz, Brentwood
Zephyr Grill, Brentwood
7 Mile House, Brisbane
AC Hotel by Marriott San Francisco Airport/Oyster Point Waterfront, Brisbane
Amami San, Brisbane
Amoura, Brisbane
Antigua Coffee Shop, Brisbane
BEN TRE Vietnamese Homestyle Cuisine, Brisbane
Bayview Park, Brisbane
Bitters + Bottles, Brisbane
Brisbane Park, Brisbane
Cafe Bunn Mi, Brisbane
Cafe de Casa, Brisbane
Centennial Dog Park, Brisbane
Costco Business Center, Brisbane
Crocker Amazon Soccer Fields, Brisbane
Darby Dan's Sandwiches Co, Brisbane
Dead Eye Coffee Bar, Brisbane
Double

John D. Morgan Park, Campbell
Jonty's Vietnamese Eatery, Campbell
Khartoum, Campbell
La Pizzeria Italian, Campbell
Ladera Grill, Campbell
Little Lou's BBQ, Campbell
Living Room Coffee Craft, Campbell
Los Gatos Creek Dog Park, Campbell
Los Gatos Creek Park, Campbell
Los Gatos Creek Trail, Campbell
Luigi's Pizza and Pasta, Campbell
Manresa Bread, Campbell
Maple Leaf Donuts, Campbell
Massage Envy - Morgan Hill, Campbell
Maurizio's, Campbell
Mod Pizza, Campbell
Morgan Hill Farmers Market, Campbell
Naschmarkt, Campbell
Nick The Greek, Campbell
Noah's Bar & Bistro, Campbell
Odeum, Campbell
Opa, Campbell
Oralé Fish Tacos, Campbell
Orchard City Kitchen, Campbell
Peet's Coffee & Tea, Campbell
Psycho Donuts, Campbell
Rancho Cañada Del Oro OSP, Campbell
Recycle Bookstore, Campbell
Red Robin Gourmet Burgers and Brews, Campbell
Regal Bagel, Campbell
Residence Inn San Jose South/Morgan Hill, Campbell
Rocket Fizz, Campbell
Rosy's at the Beach, Campbell
Santa Teresa Golf Course, Campbell
Siam Thai Res

...

River Inn Grill, Santa Rosa
Rivers End Restaurant, Jenner CA, Santa Rosa
Rosso Pizzeria and Wine Bar, Santa Rosa
Rosso Rosticceria + Eventi, Santa Rosa
Roxy Stadium 14, Santa Rosa
Royal China Restaurant, Santa Rosa
Russian River Brewing Company, Santa Rosa
Russian River Pub, Santa Rosa
Russian River Valley, Santa Rosa
SEA Noodle Bar, Santa Rosa
SEA Thai Bistro, Santa Rosa
Safeway, Santa Rosa
Sam's For Play Cafe & Catering, Santa Rosa
Santa Rosa Creek Trail, Santa Rosa
Santa Rosa Farmers Market, Santa Rosa
Santa Rosa Seafood, Santa Rosa
Santa Rosa Seafood Raw Bar & Grill, Santa Rosa
Sazón - Peruvian Cuisine, Santa Rosa
Seaside Metal, Santa Rosa
Sequoia Floral International, Santa Rosa
Sift Cupcake & Dessert Bar, Santa Rosa
Snoopy's Gallery And Giftshop, Santa Rosa
Snoopy's Home Ice, Santa Rosa
Sonoma Canopy Tours, Santa Rosa
Sonoma County Family YMCA, Santa Rosa
Sonoma Valley Bagel, Santa Rosa
Spinster Sisters, Santa Rosa
Spring Lake Park, Santa Rosa
St. Francis Winery & Vineyards, Sant

Yee Tock Chee Park, Sausalito
38º North Lounge, Sebastopol
Andy's Produce, Sebastopol
Antique Society, Sebastopol
Arrowood Vineyards & Winery, Sebastopol
BBQ Smokehouse Bistro & Catering, Sebastopol
Balletto Vineyards & Winery, Sebastopol
Beekind, Sebastopol
Benziger Family Winery, Sebastopol
Cafe La Haye, Sebastopol
Cochon Volant BBQ, Sebastopol
Community Market, Sebastopol
Copperfield's Books, Sebastopol
Crooked Goat Brewing, Sebastopol
El Coronel Mexican Restaurant, Sebastopol
El Dorado Kitchen, Sebastopol
El Molino Central, Sebastopol
Emeritus Vineyards, Sebastopol
Enoteca Della Santina, Sebastopol
Fairmont Sonoma Mission Inn & Spa, Sebastopol
Fandee's, Sebastopol
Fern Bar, Sebastopol
Fork Roadhouse, Sebastopol
Gaige House, Sebastopol
Garden Court Cafe, Sebastopol
Glen Ellen Star, Sebastopol
Glen Ellen Village Market, Sebastopol
Graton, CA, Sebastopol
Gravenstein Grill, Sebastopol
Gypsy Cafe, Sebastopol
Hamel Family Wines, Sebastopol
Handline, Sebastopol
Hardcore Espresso, Sebastop

United Concrete Pumping, South San Francisco
United States Postal Service, South San Francisco
Vista 205 Beauty, South San Francisco
White Elephant Restaurant, South San Francisco
Windy Hill Open Space Preserve, South San Francisco
Windy Hill Summit, South San Francisco
Woodside Store, California Historical Landmark No 93, South San Francisco
Wunderlich County Park, South San Francisco
Zumiez, South San Francisco
firehouse bistro, South San Francisco
Acacia House, St. Helena
Acme Fine Wines, St. Helena
Anomaly Vineyards, St. Helena
Archetype, St. Helena
Auberge du Soleil, St. Helena
Azteca Market, St. Helena
Ballentine Vineyards, St. Helena
Beringer Vineyards, St. Helena
Brasswood Bar + Kitchen, St. Helena
Cameo Cinema, St. Helena
Charles Krug Winery, St. Helena
Cook St. Helena, St. Helena
Crane Park, St. Helena
Culinary Institute of America at Greystone, St. Helena
Failla Wines, St. Helena
Farmstead at Long Meadow Ranch, St. Helena
Freemark Abbey Winery, St. Helena
Frog's Leap Winery,

Whole Foods Market, Sunnyvale
Wingstop, Sunnyvale
YAYOI, Sunnyvale
Yoga@Cindy's, Sunnyvale
Z Pizza, Sunnyvale
sweetgreen, Sunnyvale
A Taste of Rome Caffe, Tiburon
Angel Island State Park, Tiburon
Aurora Ristorante Italiano, Tiburon
Bar Bocce, Tiburon
Barrel House Tavern, Tiburon
Bay Area Discovery Museum, Tiburon
Blackie's Pasture Park, Tiburon
Cavallo Point, Tiburon
Cavallo Point Lodge, Tiburon
Copita Tequileria y Comida, Tiburon
Dr Lyford's Stone Tower, Tiburon
Driver's Market, Tiburon
Elephant rock, Tiburon
Equator, Tiburon
Fish., Tiburon
Fort Baker, Tiburon
Gabrielson Park, Tiburon
Hamburgers, Tiburon
Heath Ceramics, Tiburon
Joinery, Tiburon
Lappert's Ice Cream, Tiburon
Le Garage, Tiburon
Luna Blu, Tiburon
Mt Livermore, Tiburon
Old Rail Trail, Tiburon
Osteria Divino, Tiburon
Paradise Beach Park, Tiburon
Plaza Viña Del Mar, Tiburon
Poggio, Tiburon
Point Tiburon Plaza, Tiburon
Rustic Bakery, Tiburon
Salito's Crab House & Prime Rib, Tiburon
Sam's Anchor Cafe, Tiburon
San Drino, Tiburo

Flywheel Sports - Walnut Creek, Walnut Creek
Fresh Millions, Walnut Creek
Gardens at Heather Farm, Walnut Creek
Genova Delicatessen, Walnut Creek
Gott’s Roadside, Walnut Creek
Heather Farm Dog Park, Walnut Creek
Heather Farm Park, Walnut Creek
Heather Farm Park Playground, Walnut Creek
Heather Farm Park Swim Center, Walnut Creek
Heather Farms Softball Fields 3 & 4, Walnut Creek
House of Sake, Walnut Creek
Ice Monster, Walnut Creek
Ike's Sandwiches, Walnut Creek
In-N-Out Burger, Walnut Creek
Iron Horse Regional Trail, Walnut Creek
Iron Horse Trail, Walnut Creek
Jamba Juice, Walnut Creek
Jimmy John's, Walnut Creek
Katy's Korner, Walnut Creek
Kinder's Meats Deli BBQ, Walnut Creek
Lafayette-Moraga Regional Trail, Walnut Creek
Leonidas Belgian Chocolates, Walnut Creek
Lesher Center for the Arts, Walnut Creek
Lettuce, Walnut Creek
Limón Peruvian Rotisserie, Walnut Creek
Lokanta, Walnut Creek
Lottie's Creamery, Walnut Creek
Massage Envy - Walnut Creek, Walnut Creek
Mendocino Farms, Walnut Cre

In [25]:
for key in dist_dic.keys():
    for venue, distance in dist_dic[(key)].items():
        ven_per_area.loc[(key),(f'{venue[1]}, {venue[0]}')] = distance

In [26]:
ven_per_area.head()

lat_venue  long_venue  \
area    name                                                    
Alameda Alameda Beach                   37.760926 -122.269280   
        Alameda Dog Park                37.767796 -122.272758   
        Alameda Farmers' Market         37.774196 -122.276047   
        Alameda Island Brewing Company  37.768483 -122.238984   
        Alameda Theatre & Cineplex      37.764680 -122.243946   

                                        Alameda Beach, Alameda  \
area    name                                                     
Alameda Alameda Beach                                 0.000000   
        Alameda Dog Park                            821.784781   
        Alameda Farmers' Market                    1588.920053   
        Alameda Island Brewing Company             2798.105826   
        Alameda Theatre & Cineplex                 2270.813570   

                                        Alameda Dog Park, Alameda  \
area    name                                                        
Alameda Alameda Beach                                  821.784781   
        Alameda Dog Park                                 0.000000   
        Alameda Farmers' Market                        767.140717   
        Alameda Island Brewing Company                2976.731607   
        Alameda Theatre & Cineplex                    2562.075404   

                                        Alameda Farmers' Market, Alameda  \
area    name                                                               
Alameda Alameda Beach                                        1588.920053   
        Alameda Dog Park                                      767.140717   
        Alameda Farmers' Market                                 0.000000   
        Alameda Island Brewing Company                       3326.440564   
        Alameda Theatre & Cineplex                           3019.117118   

                                        Alameda Island Brewing Company, Alameda  \
area    name                                                                      
Alameda Alameda Beach                                               2798.105826   
        Alameda Dog Park                                            2976.731607   
        Alameda Farmers' Market                                     3326.440564   
        Alameda Island Brewing Company                                 0.000000   
        Alameda Theatre & Cineplex                                   607.752879   

                                        Alameda Theatre & Cineplex, Alameda  \
area    name                                                                  
Alameda Alameda Beach                                           2270.813570   
        Alameda Dog Park                                        2562.075404   
        Alameda Farmers' Market                                 3019.117118   
        Alameda Island Brewing Company                           607.752879   
        Alameda Theatre & Cineplex                                 0.000000   

                                        American Oak, Alameda  \
area    name                                                    
Alameda Alameda Beach                             2425.678094   
        Alameda Dog Park                          2682.212226   
        Alameda Farmers' Market                   3107.351510   
        Alameda Island Brewing Company             429.877200   
        Alameda Theatre & Cineplex                 178.158077   

                                        Angel Fish Japanese Cuisine, Alameda  \
area    name                                                                   
Alameda Alameda Beach                                            3658.303717   
        Alameda Dog Park                                         4427.507383   
        Alameda Farmers' Market                                  5161.308588   
        Alameda Island Brewing Company                           3442.732811   
        Alameda Theatre & Cinepl

In [27]:
ven_per_area.to_csv("../data/distance_df.csv")

#### Average Distance per Category per Area

In [28]:
cat_per_area = pd.merge(ven_per_area,
                        distance_df,
                        on = ["lat_venue","long_venue","area","name"],
                        how = "left"
                       )

In [29]:
cat_per_area.drop(columns=["lat_venue","long_venue"],inplace = True)

In [30]:
cat_df = distance_df[["name","categories","area","id"]].drop_duplicates()

In [31]:
cat_df["va"] = cat_df["name"] + ", " + cat_df["area"]

In [32]:
cat_df["ac"] = cat_df["area"] + ", " + cat_df["categories"]

In [33]:
cat_df.head()

,name,categories,area,id,va,ac
0,Chuck Corica Golf Complex,Golf Course,Alameda,Chuck Corica Golf ComplexAlamedaGolf Course0 ...,"Chuck Corica Golf Complex, Alameda","Alameda, Golf Course"
1,Shoreline Park,Park,Alameda,Shoreline ParkAlamedaPark0 37.738885\n1...,"Shoreline Park, Alameda","Alameda, Park"
2,Angel Fish Japanese Cuisine,Japanese Restaurant,Alameda,Angel Fish Japanese CuisineAlamedaJapanese Res...,"Angel Fish Japanese Cuisine, Alameda","Alameda, Japanese Restaurant"
3,La Penca Azul,Mexican Restaurant,Alameda,La Penca AzulAlamedaMexican Restaurant0 ...,"La Penca Azul, Alameda","Alameda, Mexican Restaurant"
4,La Val's Pizza,Italian Restaurant,Alameda,La Val's PizzaAlamedaItalian Restaurant0 ...,"La Val's Pizza, Alameda","Alameda, Italian Restaurant"


In [34]:
cat_dict = {}
for i in cat_df["id"].unique():
    cat_dict[cat_df[cat_df["id"] == i]["va"].values[0]] = cat_df[cat_df["id"] == i]["categories"].values[0]

In [35]:
cat_per_area.rename(columns=cat_dict,inplace = True)

In [36]:
cat_per_area.head()

,area,name,Beach,Dog Run,Farmers Market,Brewery,Multiplex,Bar,Japanese Restaurant,Middle Eastern Restaurant,...,Vineyard,Wine Bar,Hotel,Bed & Breakfast,Golf Course,Playground,Park,New American Restaurant,categories,id
0,Alameda,Alameda Beach,0.000000,821.784781,1588.920053,2798.105826,2270.813570,2425.678094,3658.303717,2692.288018,...,66760.475296,71605.334771,71392.954351,71763.844737,70891.784746,72284.501436,71136.036606,71277.157282,Beach,Alameda BeachAlamedaBeach0 37.738885\n1...
1,Alameda,Alameda Dog Park,821.784781,0.000000,767.140717,2976.731607,2562.075404,2682.212226,4427.507383,2900.819646,...,65975.660549,70813.424249,70601.549095,70971.397667,70100.588595,71491.237364,70344.906825,70486.055265,Dog Run,Alameda Dog ParkAlamedaDog Run0 37.7388...
2,Alameda,Alameda Farmers' Market,1588.920053,767.140717,0.000000,3326.440564,3019.117118,3107.351510,5161.308588,3277.797006,...,65245.075743,70075.893601,69864.511379,70233.344150,69363.758785,70752.385190,69608.139188,69749.313270,Farmers Market,Alameda Farmers' MarketAlamedaFarmers Market0 ...
3,Alameda,Alameda Island Brewing Company,2798.105826,2976.731607,3326.440564,0.000000,607.752879,429.877200,3442.732811,140.470442,...,66201.413647,71126.122176,70907.611691,71291.256509,70404.258870,71821.901894,70647.489767,70788.152353,Brewery,Alameda Island Brewing CompanyAlamedaBrewery0 ...
4,Alameda,Alameda Island Brewing Company,2798.105826,2976.731607,3326.440564,0.000000,607.752879,429.877200,3442.732811,140.470442,...,66201.413647,71126.122176,70907.611691,71291.256509,70404.258870,71821.901894,70647.489767,70788.152353,Brewery,Alameda Island Brewing CompanyAlamedaBrewery0 ...


In [37]:
cat_per_area.drop(columns=["id"],inplace = True)

In [38]:
cat_per_area.head()

,area,name,Beach,Dog Run,Farmers Market,Brewery,Multiplex,Bar,Japanese Restaurant,Middle Eastern Restaurant,...,Garden,Vineyard,Wine Bar,Hotel,Bed & Breakfast,Golf Course,Playground,Park,New American Restaurant,categories
0,Alameda,Alameda Beach,0.000000,821.784781,1588.920053,2798.105826,2270.813570,2425.678094,3658.303717,2692.288018,...,71901.614403,66760.475296,71605.334771,71392.954351,71763.844737,70891.784746,72284.501436,71136.036606,71277.157282,Beach
1,Alameda,Alameda Dog Park,821.784781,0.000000,767.140717,2976.731607,2562.075404,2682.212226,4427.507383,2900.819646,...,71108.905234,65975.660549,70813.424249,70601.549095,70971.397667,70100.588595,71491.237364,70344.906825,70486.055265,Dog Run
2,Alameda,Alameda Farmers' Market,1588.920053,767.140717,0.000000,3326.440564,3019.117118,3107.351510,5161.308588,3277.797006,...,70370.595751,65245.075743,70075.893601,69864.511379,70233.344150,69363.758785,70752.385190,69608.139188,69749.313270,Farmers Market
3,Alameda,Alameda Island Brewing Company,2798.105826,2976.731607,3326.440564,0.000000,607.752879,429.877200,3442.732811,140.470442,...,71432.237020,66201.413647,71126.122176,70907.611691,71291.256509,70404.258870,71821.901894,70647.489767,70788.152353,Brewery
4,Alameda,Alameda Island Brewing Company,2798.105826,2976.731607,3326.440564,0.000000,607.752879,429.877200,3442.732811,140.470442,...,71432.237020,66201.413647,71126.122176,70907.611691,71291.256509,70404.258870,71821.901894,70647.489767,70788.152353,Brewery


In [39]:
cat_per_area["ac"] = cat_per_area["area"] + ", " + cat_per_area["categories"]

In [40]:
cat_per_area = pd.DataFrame(cat_per_area.groupby(["ac","name"]).min())

In [41]:
cat_per_area_t = cat_per_area.T

In [42]:
# Source: https://stackoverflow.com/questions/40311987/pandas-mean-of-columns-with-the-same-names

cat_per_area_t = cat_per_area_t.groupby(by=cat_per_area_t.index, axis = 0).apply(lambda g: g.min() if isinstance(g.iloc[0,0], numbers.Number) else g.iloc[0])

In [43]:
cat_per_area = cat_per_area_t.T

In [44]:
cat_per_area.reset_index(inplace = True)

In [45]:
cat_per_area = pd.merge(cat_per_area,
                        cat_df[["area","categories","ac"]],
                        on = ["ac"],
                        how = "left"
                       )

In [49]:
cat_per_area.drop(columns = ["area_y","categories_y"], inplace = True)
cat_per_area.rename(columns={"area_x":"area","categories_x":"categories"}, inplace = True)

In [50]:
cat_per_area.set_index(["area","categories"], inplace = True)

In [51]:
cat_per_area.drop(columns="ac", inplace = True)

In [52]:
cat_per_area.head()

name           ATM    Acai House  \
area    categories                                                             
Alameda Afghan Restaurant      Q's Halal Chicken  19511.372574  13979.911196   
        Airport Service          JSX (JetsuiteX)  14136.570906  18560.157257   
        American Restaurant  Scolari's Good Eats  19382.759806  13993.863443   
        Arcade                High Scores Arcade  19432.761716  14100.866265   
        Arcade                High Scores Arcade  19432.761716  14100.866265   

                            Accessories Store Acupuncturist Afghan Restaurant  \
area    categories                                                              
Alameda Afghan Restaurant        11198.426865  62759.515967               0.0   
        Airport Service          16007.462666  57497.347395       5382.747523   
        American Restaurant      11400.194018  62593.193753        211.060273   
        Arcade                   11207.098384  62707.548759        126.411934   
        Arcade                   11207.098384  62707.548759        126.411934   

                            African Restaurant       Airport Airport Lounge  \
area    categories                                                            
Alameda Afghan Restaurant         13365.306366  15743.376839   20565.965024   
        Airport Service           16368.791699  10389.712669   20029.347618   
        American Restaurant       13278.261825  15621.447513   20378.449515   
        Arcade                     13489.35138  15661.094996   20648.784776   
        Arcade                     13489.35138  15661.094996   20648.784776   

                            Airport Service  ...    Whisky Bar      Windmill  \
area    categories                           ...                               
Alameda Afghan Restaurant       4889.294047  ...  14412.533189  23410.654951   
        Airport Service                 0.0  ...  18910.681456  27228.777586   
        American Restaurant     4751.309024  ...  14420.111469  23379.105276   
        Arcade                  4818.601367  ...  14534.531291  23536.633209   
        Arcade                  4818.601367  ...  14534.531291  23536.633209   

                                Wine Bar    Wine Shop        Winery  \
area    categories                                                    
Alameda Afghan Restaurant    4898.605383   417.555842    6208.45614   
        Airport Service      1858.348631  5071.378933  11196.764306   
        American Restaurant  5051.895002   467.336745   6249.937742   
        Arcade               4964.247571   296.311477   6324.153683   
        Arcade               4964.247571   296.311477   6324.153683   

                              Wings Joint Women's Store  Yoga Studio  \
area    categories                                                     
Alameda Afghan Restaurant     8257.048208  12565.688209  2466.149939   
        Airport Service      13515.535041  17800.759025  4819.403358   
        American Restaurant   8433.407727  12742.219449  2556.344207   
        Arcade                8301.361417   12609.25216  2341.560429   
        Arcade                8301.361417   12609.25216  2341.560429   

                                     Zoo  Zoo Exhibit  
area    categories                                     
Alameda Afghan Restaurant    8707.117751  8721.235556  
        Airport Service      5803.332714  5830.343855  
        American Restaurant  8710.481353  8726.492864  
        Arcade               8584.193959   8598.04952  
        Arcade               8584.193959   8598.04952  

[5 rows x 428 columns]

### Category Frequency per Area

In [53]:
# Inspiration: https://towardsdatascience.com/store-locations-d1025df22865

In [54]:
freq_cat = capstone_df[["area","categories","name","lat_venue","long_venue"]]

In [55]:
freq_cat["id"] = freq_cat["name"] + freq_cat["area"] + freq_cat["categories"] + str(freq_cat["lat_venue"]) + str(freq_cat["long_venue"])

<ipython-input-55-ce53334408a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freq_cat["id"] = freq_cat["name"] + freq_cat["area"] + freq_cat["categories"] + str(freq_cat["lat_venue"]) + str(freq_cat["long_venue"])


In [56]:
cat_total = pd.DataFrame(freq_cat.groupby(["categories"])["id"].count())

In [57]:
cat_total.head()

,id
categories,
ATM,1
Acai House,1
Accessories Store,15
Acupuncturist,1
Afghan Restaurant,20


In [58]:
freq_cat = pd.DataFrame(freq_cat.groupby(["area","categories"])["id"].count())

In [59]:
freq_cat.head()

id
area    categories             
Alameda Afghan Restaurant     1
        Airport Service       1
        American Restaurant   2
        Arcade                3
        Bakery                3

In [60]:
cat_dic = {}

for area in freq_cat.index.get_level_values("area").unique():
    cat_dic[area] = [venue for venue in freq_cat[freq_cat.index.get_level_values("area") == area].index.get_level_values("categories").unique()]

In [61]:
freq_cat["freq"] = 0

In [62]:
for k,v in cat_dic.items():
    for category in v:
        freq_cat.loc[(k,category),"freq"] = freq_cat[(freq_cat.index.get_level_values("area") == k) &
                                                     (freq_cat.index.get_level_values("categories") == category)]["id"].values[0] / cat_total[cat_total.index.get_level_values("categories") == category]["id"].values[0]

In [63]:
freq_cat.drop(columns = "id", inplace = True)

### Final Dataset

In [64]:
merged_df = pd.merge(cat_per_area,
                    user_as_pct_pop,
                    on = ["area","categories"],
                    how = "left"
                   )


final_df = pd.merge(merged_df,
                    freq_cat,
                    on = ["area","categories"],
                    how = "left"
                   )

In [65]:
final_df.head()

name           ATM    Acai House  \
area    categories                                                             
Alameda Afghan Restaurant      Q's Halal Chicken  19511.372574  13979.911196   
        Airport Service          JSX (JetsuiteX)  14136.570906  18560.157257   
        American Restaurant  Scolari's Good Eats  19382.759806  13993.863443   
        Arcade                High Scores Arcade  19432.761716  14100.866265   
        Arcade                High Scores Arcade  19432.761716  14100.866265   

                            Accessories Store Acupuncturist Afghan Restaurant  \
area    categories                                                              
Alameda Afghan Restaurant        11198.426865  62759.515967               0.0   
        Airport Service          16007.462666  57497.347395       5382.747523   
        American Restaurant      11400.194018  62593.193753        211.060273   
        Arcade                   11207.098384  62707.548759        126.411934   
        Arcade                   11207.098384  62707.548759        126.411934   

                            African Restaurant       Airport Airport Lounge  \
area    categories                                                            
Alameda Afghan Restaurant         13365.306366  15743.376839   20565.965024   
        Airport Service           16368.791699  10389.712669   20029.347618   
        American Restaurant       13278.261825  15621.447513   20378.449515   
        Arcade                     13489.35138  15661.094996   20648.784776   
        Arcade                     13489.35138  15661.094996   20648.784776   

                            Airport Service  ...     Wine Bar    Wine Shop  \
area    categories                           ...                             
Alameda Afghan Restaurant       4889.294047  ...  4898.605383   417.555842   
        Airport Service                 0.0  ...  1858.348631  5071.378933   
        American Restaurant     4751.309024  ...  5051.895002   467.336745   
        Arcade                  4818.601367  ...  4964.247571   296.311477   
        Arcade                  4818.601367  ...  4964.247571   296.311477   

                                   Winery   Wings Joint Women's Store  \
area    categories                                                      
Alameda Afghan Restaurant      6208.45614   8257.048208  12565.688209   
        Airport Service      11196.764306  13515.535041  17800.759025   
        American Restaurant   6249.937742   8433.407727  12742.219449   
        Arcade                6324.153683   8301.361417   12609.25216   
        Arcade                6324.153683   8301.361417   12609.25216   

                             Yoga Studio          Zoo  Zoo Exhibit  \
area    categories                                                   
Alameda Afghan Restaurant    2466.149939  8707.117751  8721.235556   
        Airport Service      4819.403358  5803.332714  5830.343855   
        American Restaurant  2556.344207  8710.481353  8726.492864   
        Arcade               2341.560429  8584.193959   8598.04952   
        Arcade               2341.560429  8584.193959   8598.04952   

                            user_rate_as_pct_pop      freq  
area    categories                                          
Alameda Afghan Restaurant               0.007021  0.050000  
        Airport Service                 0.000000  0.111111  
        American Restaurant             0.006995  0.005222  
        Arcade                          0.000000  0.428571  
        Arcade                          0.000000  0.428571  

[5 rows x 430 columns]

In [66]:
final_df["venues_per_capita"] = 0

for index in final_df.index:
    final_df.loc[index, "venues_per_capita"] = ven_per_cap[ven_per_cap.index.get_level_values("area") == index[0]]["venues_per_capita"].values[0]

/Users/akhairunnisa/miniforge3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2898: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
/Users/akhairunnisa/miniforge3/lib/python3.9/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [67]:
final_df["cs_per_capita"] = 0

for index in final_df.index:
    if index[0] in cs_per_cap.index.get_level_values("area").unique():
        final_df.loc[index, "cs_per_capita"] = cs_per_cap[cs_per_cap.index.get_level_values("area") == index[0]]["cs_per_capita"].values[0]
    else:
        0

In [68]:
final_df = pd.merge(final_df,
                    capstone_df[["area","categories","name","price_level","rating","user_ratings_total"]],
                    on = ["area","categories","name"],
                    how = "left"
                   )

In [69]:
final_df.set_index(["area","categories","name"], inplace = True)
final_df.head()

ATM    Acai House  \
area    categories          name                                              
Alameda Afghan Restaurant   Q's Halal Chicken    19511.372574  13979.911196   
        Airport Service     JSX (JetsuiteX)      14136.570906  18560.157257   
        American Restaurant Scolari's Good Eats  19382.759806  13993.863443   
                            Scolari's Good Eats  19382.759806  13993.863443   
        Arcade              High Scores Arcade   19432.761716  14100.866265   

                                                Accessories Store  \
area    categories          name                                    
Alameda Afghan Restaurant   Q's Halal Chicken        11198.426865   
        Airport Service     JSX (JetsuiteX)          16007.462666   
        American Restaurant Scolari's Good Eats      11400.194018   
                            Scolari's Good Eats      11400.194018   
        Arcade              High Scores Arcade       11207.098384   

                                                Acupuncturist  \
area    categories          name                                
Alameda Afghan Restaurant   Q's Halal Chicken    62759.515967   
        Airport Service     JSX (JetsuiteX)      57497.347395   
        American Restaurant Scolari's Good Eats  62593.193753   
                            Scolari's Good Eats  62593.193753   
        Arcade              High Scores Arcade   62707.548759   

                                                Afghan Restaurant  \
area    categories          name                                    
Alameda Afghan Restaurant   Q's Halal Chicken                 0.0   
        Airport Service     JSX (JetsuiteX)           5382.747523   
        American Restaurant Scolari's Good Eats        211.060273   
                            Scolari's Good Eats        211.060273   
        Arcade              High Scores Arcade         126.411934   

                                                African Restaurant  \
area    categories          name                                     
Alameda Afghan Restaurant   Q's Halal Chicken         13365.306366   
        Airport Service     JSX (JetsuiteX)           16368.791699   
        American Restaurant Scolari's Good Eats       13278.261825   
                            Scolari's Good Eats       13278.261825   
        Arcade              High Scores Arcade         13489.35138   

                                                      Airport Airport Lounge  \
area    categories          name                                               
Alameda Afghan Restaurant   Q's Halal Chicken    15743.376839   20565.965024   
        Airport Service     JSX (JetsuiteX)      10389.712669   20029.347618   
        American Restaurant Scolari's Good Eats  15621.447513   20378.449515   
                            Scolari's Good Eats  15621.447513   20378.449515   
        Arcade              High Scores Arcade   15661.094996   20648.784776   

                                                Airport Service  \
area    categories          name                                  
Alameda Afghan Restaurant   Q's Halal Chicken       4889.294047   
        Airport Service     JSX (JetsuiteX)                 0.0   
        American Restaurant Scolari's Good Eats     4751.309024   
                            Scolari's Good Eats     4751.309024   
        Arcade              High Scores Arcade      4818.601367   

                                                Alternative Healer  ...  \
area    categories          name                                    ...   
Alameda Afghan Restaurant   Q's Halal Chicken         17828.958588  ...   
        Airport Service     JSX (JetsuiteX)           22301.288157  ...   
        American Restaurant Scolari's Good Eats       17837.861334  ...   
                            Scolari's Good Eats       17837.861334  ...   
        Arcade              High Scores Arcade        17950.692116  ...   

                                

In [70]:
capstone_df.drop_duplicates(inplace = True)

In [71]:
final_df.to_csv("../data/final_df_min.csv")